In [1]:
import pymysql
from datetime import datetime

In [2]:
your_sql_password = "yourPassword"

cnx = pymysql.connect(host='localhost', user='root', password=your_sql_password,
             db='airbnb',charset='utf8mb4',cursorclass=pymysql.cursors.DictCursor )

In [4]:
def work():
    
    today = datetime.strptime('2022-12-25', '%Y-%m-%d')
    todaystr = '2022-12-25'
    
    ## Logging in or New Account
    cont = False 
    while cont == False:
        choice = input("Please enter either 'login' or 'new user'")
        choice = choice.lower()
        if choice == 'login' or choice == 'new user':
            cont = True
    ### Loging in
    succ = False
    if choice == 'login':
        att = 0
        
        while succ == False and att<5:
            in_username = input("Enter your username: ")
            in_password = input("Enter your password: ")
        
            stmt = "CALL checkAccount( '" + in_username + "', '" + in_password + "')"
            cur = cnx.cursor()
            cur.execute(stmt)
            
            for row in cur.fetchall():
                if row == {'COUNT(1)': 1}:
                    succ=True
                else:
                    print("Username/Password combination not found")
                    att+=1
            cur.close()
    ## New account login
    if succ == False or choice == "new user":
        if succ == False:
            print("Too many failed attempts, please create new account")
        ## New unique username
        succ = False
        while succ == False:
            in_username = input("Please enter a unique username (max 32 char): ")
            
            stmt = "CALL checkUser( '" + in_username + "')"
            cur = cnx.cursor()
            cur.execute(stmt)
            
            for row in cur.fetchall():
                if row == {'COUNT(1)': 1}:
                    print("Username already exists, please enter a different username")
                else:
                    if len(in_username)<=32 and len(in_username)>0:
                        succ=True
                    else:
                        print("Incorrect formatting, enter a username <= 32 characters")
        ## New password
        succ = False
        while succ == False:
            in_password = input("Please enter a password (max 32 char): ")
            if len(in_password)<=32 and len(in_password)>0:
                succ=True
            else:
                print("Incorrect formatting, please enter a password <= 32 characters")
                
        ## New Name
        succ = False
        while succ == False:
            in_name = input("Please enter your name (max 64 char): ")
            if len(in_name)<=64 and len(in_name)>0:
                succ=True
            else:
                print("Incorrect formatting, please enter a name <= 64 characters")
                
        ## New email
        succ = False
        while succ == False:
            in_email = input("Please enter your email (max 64 char): ")
            if len(in_email)<=64 and len(in_email)>0:
                succ=True
            else:
                print("Incorrect formatting, please enter an email <= 64 characters")
                
        ## New phone
        succ = False
        while succ == False:
            in_phone = input("Please enter your phone (Format: xxxxxxxxxx): ")
            if len(in_phone)!=10:
                print("Incorrect formatting, please only type the 10 digits (xxxxxxxxxx)")
            elif in_phone.isdigit() == False:
                print("Incorrect formatting, please type a valid phone number (xxxxxxxxxx)")
            else:
                succ=True
                
        ##insert new user
        inserting = (in_username, in_password, in_name, in_email, in_phone)
        stmt = "INSERT INTO user (username, password, name, email, phone) VALUES (%s,%s,%s,%s,%s)"
        cur = cnx.cursor()
        cur.execute(stmt, inserting)
        cnx.commit()
        cur.close()
        
        
    logout = False
    
    while logout == False:
        
        choice = input("Please enter 'logout', 'my transactions', 'my account', or 'search'")
        
        if choice == 'logout':
            logout = True
            
        elif choice == 'my transactions':
            back = False
            while back == False:
                stmt = "CALL userTransactions('"+ in_username + "')"
                cur = cnx.cursor()
                cur.execute(stmt)
                
                added = 0
                
                print("")
                print("Your transaction history:")
                for row in cur.fetchall():
                    added +=1
                    print("")
                    print("Transaction ID:", row["transactionid"])
                    print("Date:", row["startdate"], "to", row["enddate"])
                    print("Address:", row["street"], row["town_name"] + ",", row["state"], row["zip"])
                    
                cur.close()
                
                
                if added == 0:
                    print("You have no transactions")
                    back = True 
                    break
                
                    
                choice = input("Please enter 'make review', 'delete transaction', or 'back'")

                if choice == 'back':
                    back = True

                elif choice == 'make review':
                    ## check to see if they have passed date and if they havent reviewed yet, 
                    ## if so then can review
                    stmt = "CALL reviewableTransactions('"+ in_username + "', '" + todaystr +"')"
                    cur = cnx.cursor()
                    cur.execute(stmt)
                    
                    print("")
                    print("Reviewable transactions:")
                    added = 0
                    possible = {}
                    for row in cur.fetchall():
                        added +=1
                        print("")
                        print("Transaction ID:", row["transactionid"])
                        print("Date:", row["startdate"], "to", row["enddate"])
                        print("Address:", row["street"], row["town_name"] + ",", row["state"], row["zip"])
                        possible[str(row["transactionid"])] = 1
                    cur.close()
                    
                    if added == 0: 
                        print("There are no transactions you can review. Only transactions that have already completed and are not yet reviewed can be reviewed.")
                        break
                        
                    review_id = input("Please enter the id of the transaction you would like to review:")
                    
                    if review_id not in possible:
                        print("This is not a valid id to review")
                    else:
                        cont= True
                        review_txt = input("Please enter your review (max len = 1000 characters):")
                        if len(review_txt) >= 1000 or len(review_txt) == 0:
                            print("Not a valid review length")
                            cont=False
                            
                        review_rating = input("Please enter your rating (0 to 5):")
                        if review_rating not in ['0','1','2','3','4','5']:
                            print("Not a valid rating")
                            cont = False
                            
                        if cont == True:
                            inserting = (int(review_id), int(review_rating), review_txt)
                            stmt = "INSERT INTO review (transactionid, rating, review) VALUES (%s,%s,%s)"
                            cur = cnx.cursor()
                            cur.execute(stmt, inserting)
                            cnx.commit()
                            cur.close()
                    
                    
                            back = True
                elif choice == 'delete transaction':
                    ## check to see if they have passed date, if not then can delete
                    stmt = "CALL deletableTransactions('"+ in_username + "', '" + todaystr +"')"
                    cur = cnx.cursor()
                    cur.execute(stmt)
                    
                    print("")
                    print("Cancellable transactions:")
                    added = 0
                    possible = {}
                    for row in cur.fetchall():
                        added +=1
                        print("")
                        print("Transaction ID:", row["transactionid"])
                        print("Date:", row["startdate"], "to", row["enddate"])
                        print("Address:", row["street"], row["town_name"] + ",", row["state"], row["zip"])
                        possible[str(row["transactionid"])] = 1
                    cur.close()
                    
                    if added == 0: 
                        print("There are no transactions you can cancel. Only transactions that have yet to happen can be cancelled.")
                        break
                    cancel_id = input("Please enter the id of the transaction you would like to cancel:")
                    
                    if cancel_id not in possible:
                        print("This is not a valid id to cancel")
                    else:
                        deleting = (cancel_id)
                        stmt = "DELETE FROM transaction WHERE transactionid = %s"
                        cur = cnx.cursor()
                        cur.execute(stmt, deleting)
                        cnx.commit()
                        cur.close()
                        
                        
                    
                    
                    
                    
        elif choice == "my account":
            
            back = False
            while back == False:
                stmt = "CALL userDetails('"+ in_username + "')"
                cur = cnx.cursor()
                cur.execute(stmt)


                for row in cur.fetchall():
                    print("")
                    print ("Name:", row['name'])
                    print ("Email:", row['email'])
                    print ("Phone:", row['phone'])
                    print("")
                cur.close()
                choice = input("Please enter 'update account', or 'back'")
                if choice == "update account":
                    choice1 = input("Do you want to update your email, or phone?")
                    if choice1 == "phone":
                        new = input("Please enter a valid phone number (format: xxxxxxxxxx)")
                        if (new.isdigit() == False) or (len(new) != 10):
                            print("Incorrect formatting (format: xxxxxxxxxx)")
                        else:
                            stmt = "UPDATE user SET phone = %s WHERE username = %s"
                            inserting = (new, in_username)
                            cur = cnx.cursor()
                            cur.execute(stmt, inserting)
                            cnx.commit()
                            cur.close()
                    elif choice1 == "email": 
                        new = input("Please enter your email (max 64 char):")
                        if len(in_email)<=64 and len(in_email)>0:
                            stmt = "UPDATE user SET email = %s WHERE username = %s"
                            inserting = (new, in_username)
                            cur = cnx.cursor()
                            cur.execute(stmt, inserting)
                            cnx.commit()
                            cur.close() 
                        else:
                            print("Please enter a valid email (max 64 characters)")
                    else:
                        print("Please choose either phone or email by typing 'phone' or 'email'")
                elif choice == "back":
                    back = True
                        
                            
        elif choice == "search":
            
            cont = False
            succ = False
            
            while cont==False:
                inStart = input("Please enter desired check in date (format: YYYY-MM-DD): ")
                try: 
                    datetime.strptime(inStart, '%Y-%m-%d')
                except:
                    print("Invalid date format, please enter date in the following format: YYYY-MM-DD")
                else:
                    succ = True
                
                if succ == True:
                    inEnd = input("Please enter desired check out date (format: YYYY-MM-DD): ")
                    try: 
                        datetime.strptime(inEnd, '%Y-%m-%d')
                    except:
                        print("Invalid date format, please enter date in the following format: YYYY-MM-DD")
                        succ = False
                if succ == True:
                    if datetime.strptime(inStart, '%Y-%m-%d') > datetime.strptime(inEnd, '%Y-%m-%d'):
                        print("Invalid start and end dates. Check in must be before check out.")
                    elif datetime.strptime(inStart, '%Y-%m-%d') < today:
                        print("Invalid start date. Must be in the future. Todays date:", today)
                    else: 
                        length = (datetime.strptime(inEnd, '%Y-%m-%d') - datetime.strptime(inStart, '%Y-%m-%d')).days + 1
                        cont = True
            
            
            print("")
            print("Available States: ")
            
            stmt = "CALL stateSearch( '" + inStart + "', '" + inEnd + "')"
            cur = cnx.cursor()
            cur.execute(stmt)
            
            avail_states = []
            for row in cur.fetchall():
                print (row['state'])
                avail_states.append(row['state'])
            cur.close()
                
            cont = True
            
            if len(avail_states)==0:
                print("No houses available for these dates")
                cont = False
            
            if cont == True:
                in_state = input("Please enter an available state (format: XX): ")
                in_state = in_state.upper()
                if in_state not in avail_states:
                    cont = False
                    print("Invalid state choice")
            
            
            if cont == True:  
                
                stmt = "CALL townSearch( '" + inStart + "', '" + inEnd + "' , '" + in_state + "')"
                cur = cnx.cursor()
                cur.execute(stmt)

                avail_towns = []
                print("")
                print("Available towns:")
                for row in cur.fetchall():
                    print (row['town_name'])
                    avail_towns.append(row['town_name'])
                cur.close()
                
                in_town = input("Please enter an available town: ")
                
                if in_town not in avail_towns:
                    print("Town not found")
                    cont = False
   
            
            if cont == True:
                
                
                while cont == True:
                
                    stmt = "CALL houseSearch( '" + inStart + "', '" + inEnd + "' , '" + in_town + "')"
                    cur = cnx.cursor()
                    cur.execute(stmt)
                    
                    costs = {}

                    avail_house = []
                    print("")
                    print("Available houses:")
                    for row in cur.fetchall():
                        print("")
                        print ("House ID:", row['houseid'])
                        print("Address:", row['street'], row['town_name'] + ", " + row['state'])
                        print("Beds:", row['beds'], "Baths:", row['baths'], "Sqft:", row['sqft'])
                        print("Nightly rate: $" + str(row['nightly_rate']))
                        print("Total cost: $" + str(row['nightly_rate'] * length) )
                        avail_house.append(row['houseid'])
                        costs[int(row['houseid'])] = str(row['nightly_rate'] * length)
                    cur.close()
                    
                    print("")

                    in_house = input("Please choose an available house id, or type back: ")
                    
                    if in_house == "back":
                        cont = False
                    
                    elif in_house.isnumeric() == True:
                        if int(in_house) in avail_house:
                            
                            ## print reviews if any, ask to book, go back, or menu
                            stmt = "CALL homeReviews(" + in_house + ")"
                            cur = cnx.cursor()
                            cur.execute(stmt)

                            count = 0
                            for row in cur.fetchall():
                                count += 1
                                print("")
                                print("Rating:", row['rating'])
                                print("Review:", row['review'])  

                            cur.close()
                            
                            if count == 0:
                                print("")
                                print("No reviews have been made for this house.")
                            
                            desc = input("Would you like to book this house for $" + str(costs[int(in_house)]) + "? Type Y or N")
                            if desc == "Y":
                                print("book house")
                                
                                stmt = "INSERT INTO transaction (username, houseid, startdate, enddate, cost) VALUES(%s,%s,%s,%s,%s)"
                                
                                inserting = (in_username, int(in_house), inStart, inEnd, int(costs[int(in_house)]))
                                cur = cnx.cursor()
                                cur.execute(stmt, inserting)
                                cnx.commit()
                                cur.close()
                                
                                cont = False
                        else:
                            print("House not available")
                            print("")
                    else:
                        print("Not a valid input (must be integer or 'back')")
                        print("")
                            
                

                #print("select house to see reviews and enter book or back to main menu")
                #print("enter back at any step to go to main menu")
            
        else:
            print("Invalid choice")
            
            
    
                
            
            
    


In [5]:
work()

Please enter either 'login' or 'new user'login
Enter your username: tcauley
Enter your password: password
Please enter 'logout', 'my transactions', 'my account', or 'search'my account

Name: Tim Cauley
Email: tcauley@gmail.com
Phone: 9999999999

Please enter 'update account', or 'back'update account
Do you want to update your email, or phone?phone
Please enter a valid phone number (format: xxxxxxxxxx)1111111111

Name: Tim Cauley
Email: tcauley@gmail.com
Phone: 1111111111

Please enter 'update account', or 'back'back
Please enter 'logout', 'my transactions', 'my account', or 'search'search
Please enter desired check in date (format: YYYY-MM-DD): 2023-01-01
Please enter desired check out date (format: YYYY-MM-DD): 2023-01-02

Available States: 
MA
NH
RI
VT
Please enter an available state (format: XX): Ma

Available towns:
Amherst
Boston
Lowell
Please enter an available town: Amherst

Available houses:

House ID: 7
Address: 40 Lincoln Rd Amherst, MA
Beds: 3 Baths: 2 Sqft: 1080
Nightly rat